In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

데이터 불러오기

In [2]:
data_path= "data"
signal_name = 'signal1'
normal_files = glob.glob(data_path+'/'+signal_name+'/normal/'+'*0.csv')
anormal_drift_files = glob.glob(data_path+'/'+signal_name+'/normal+anormal/drift/'+'*0.csv')
anormal_erratic_files = glob.glob(data_path+'/'+signal_name+'/normal+anormal/erratic/'+'*0.csv')
anormal_hardover_files = glob.glob(data_path+'/'+signal_name+'/normal+anormal/hardover/'+'*0.csv')
anormal_spike_files = glob.glob(data_path+'/'+signal_name+'/normal+anormal/spike/'+'*0.csv')
anormal_stuck_files = glob.glob(data_path+'/'+signal_name+'/normal+anormal/stuck/'+'*0.csv')

In [3]:
print(anormal_drift_files)

['data/signal1/normal+anormal/drift\\signal1_drift0.csv', 'data/signal1/normal+anormal/drift\\signal1_drift10.csv', 'data/signal1/normal+anormal/drift\\signal1_drift100.csv', 'data/signal1/normal+anormal/drift\\signal1_drift110.csv', 'data/signal1/normal+anormal/drift\\signal1_drift120.csv', 'data/signal1/normal+anormal/drift\\signal1_drift130.csv', 'data/signal1/normal+anormal/drift\\signal1_drift140.csv', 'data/signal1/normal+anormal/drift\\signal1_drift150.csv', 'data/signal1/normal+anormal/drift\\signal1_drift160.csv', 'data/signal1/normal+anormal/drift\\signal1_drift170.csv', 'data/signal1/normal+anormal/drift\\signal1_drift180.csv', 'data/signal1/normal+anormal/drift\\signal1_drift190.csv', 'data/signal1/normal+anormal/drift\\signal1_drift20.csv', 'data/signal1/normal+anormal/drift\\signal1_drift200.csv', 'data/signal1/normal+anormal/drift\\signal1_drift210.csv', 'data/signal1/normal+anormal/drift\\signal1_drift220.csv', 'data/signal1/normal+anormal/drift\\signal1_drift230.csv', 

In [4]:
def dataReader(path_names):
    data_n = pd.DataFrame() #판다스의 데이터프레임 형태로 프레임 생성
    for i in path_names:
        low_data = pd.read_csv(i)# 판다스 형태로 읽음, 한csv파일씩 읽기 떄문에 다음 라인에서 하나로 합침
        data_n = pd.concat([data_n,low_data],ignore_index=True)
    return data_n

In [5]:
normal_datas = dataReader(normal_files)
drift_datas = dataReader(anormal_drift_files)
hardover_datas = dataReader(anormal_hardover_files)
erratic_datas = dataReader(anormal_erratic_files)
spike_datas = dataReader(anormal_spike_files)
stuck_datas = dataReader(anormal_stuck_files)

In [29]:
all_data = pd.concat([drift_datas,hardover_datas,erratic_datas,spike_datas,stuck_datas],ignore_index=True)

In [49]:
for i in range(len(all_data)):
    if( all_data['error'][i] != 0):
        print(i)
        break

1962


data shape 변경

In [187]:
#입력, 출력값 크기 설정
n_time_in = 128 # 개의 데이터 입력으로 받음
ntime_out = 1 # 다음 한개의 데이터 목표값

In [188]:
#LSTM, LSTM-AE softmax
def X_to_XyLSTM_softmax(data_pd,ntime_in,ntime_out):#X변형할 시계열 데이터 n_time_in 만큼 하나의 input으로 봄, ntime_out만큼 뒤에꺼를 예측
    nsample = len(data_pd) - ntime_in -ntime_out + 1
    X_ntime = [0 for _ in range(nsample)]
    for i in range(nsample):
        X_ntime[i] = data_pd['value'][i:i+ntime_in]
    X_train = np.reshape(X_ntime,(nsample,ntime_in,1))#2차원 배열을 3차원 배열으로
    #print('X',X_train.shape)
    y_nfuture = [0 for _ in range(nsample)]
    for i in range(nsample):
        y_nfuture[i] = data_pd['error'][i+ntime_in:i+ntime_in+ntime_out]
    y_train = np.array(y_nfuture)
    #print('y',y_train.shape)
    return X_train,y_train

In [189]:
X_LSTM_train, y_LSTM_train = X_to_XyLSTM_softmax(all_data,n_time_in,ntime_out)

new_y = []
for i in range(len(y_LSTM_train)):
    if y_LSTM_train[i] == 0:
        new_y.append([1,0,0,0,0,0])
    if y_LSTM_train[i] == 1:
        new_y.append([0,1,0,0,0,0])
    if y_LSTM_train[i] == 2:
        new_y.append([0,0,1,0,0,0])
    if y_LSTM_train[i] == 3:
        new_y.append([0,0,0,1,0,0])
    if y_LSTM_train[i] == 4:
        new_y.append([0,0,0,0,1,0])
    if y_LSTM_train[i] == 5:
        new_y.append([0,0,0,0,0,1])
y_LSTM_train = np.array(new_y)  


In [190]:
print(X_LSTM_train.shape)
print(y_LSTM_train.shape)

(2015872, 128, 1)
(2015872, 6)


In [191]:
from sklearn.model_selection import train_test_split

# Your dataset (X represents your features, y represents your target variable)
X, y = X_LSTM_train, y_LSTM_train

# Split the data into training and testing sets (80% train, 20% test in this example)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
"""
#AE
def X_to_XyAE_shape(X,ntime_out): #X변형할 시계열 데이터, ntime_out만큼 뒤에꺼를 예측
    nsample = len(X) - ntime_out
    X_train = np.array(X[:nsample])
    y_train = np.array(X[ntime_out:len(X)])
    #print('X',X_train.shape)
    #print('y',y_train.shape)
    return X_train,y_train

n_time_in = 32 # 10개의 데이터 입력으로 받음
ntime_out = 1 # 다음 한개의 데이터 목표값
X_AE_train, y_AE_train = X_to_XyAE_shape(normal_datas['value'],ntime_out)

"""

model 생성

In [192]:
from tensorflow.python.keras.layers import InputLayer, Dense, LSTM
from tensorflow.python.keras.models import Sequential

#LSTM_AE 모델
LSTM_AE_model = Sequential()

LSTM_AE_model.add(LSTM(128, input_shape=(n_time_in, 1)))
LSTM_AE_model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
LSTM_AE_model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
LSTM_AE_model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
LSTM_AE_model.add(Dense(128, activation='relu',kernel_initializer='random_uniform'))
LSTM_AE_model.add(Dense(6,activation = 'softmax'))

LSTM_AE_model.compile(loss="categorical_crossentropy",optimizer='adam')

#LSTM 모델
LSTM_model = Sequential()

LSTM_model.add(LSTM(128, input_shape=(n_time_in, 1)))
LSTM_model.add(Dense(6,activation = 'softmax')) # error 의 label 이 총 6개 이므로

LSTM_model.compile(loss="categorical_crossentropy",optimizer='adam')

"""
#AE 모델
AE_model = Sequential()

AE_model.add(InputLayer(input_shape=(1,1)))
AE_model.add(Dense(128, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(128, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(ntime_out,activation='linear'))

AE_model.compile(loss="mean_squared_error",optimizer='adam')
"""

'\n#AE 모델\nAE_model = Sequential()\n\nAE_model.add(InputLayer(input_shape=(1,1)))\nAE_model.add(Dense(128, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(64, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(32, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(64, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(128, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(ntime_out,activation=\'linear\'))\n\nAE_model.compile(loss="mean_squared_error",optimizer=\'adam\')\n'

학습

In [193]:
#LSTM_AE_model.fit(X_LSTM_train, y_LSTM_train, epochs=1, batch_size=100, verbose=0)
LSTM_model.fit(X_train,y_train, epochs=1, batch_size=100, verbose=0)
#AE_model.fit(X_AE_train,y_AE_train, epochs=1, batch_size=100, verbose=0)

In [194]:
print(len(X_test))
print(len(y_test))

403175
403175


In [195]:
y_predict = LSTM_model.predict(X_test,verbose=0)

In [196]:
print(y_test[0])
print(y_predict[0])

[1 0 0 0 0 0]
[9.9030137e-01 4.1597742e-03 4.8770657e-04 5.2113767e-04 2.5198096e-04
 4.2780978e-03]


In [197]:
total_predict = []
for data1 in range(len(y_predict)):
    predict_01 = [0 for i in range(6)]
    max_index = np.argmax(y_predict[data1])
    predict_01[max_index] = 1
    total_predict.append(predict_01)
total_predict = np.array(total_predict)

In [198]:
print(y_test[0])
print(total_predict[0])

[1 0 0 0 0 0]
[1 0 0 0 0 0]


In [199]:
cnt = 0
for i in range(len(y_test)):
    result = np.array_equal(y_test[i], total_predict[i]) 
    if(result == False):
        print(i)
        cnt += 1

print("cnt",cnt)
print(len(y_test))

print((len(y_test)-cnt)/len(y_test))

1
46
77
101
102
175
182
185
189
190
227
318
419
427
441
473
498
516
524
551
599
615
616
637
660
673
674
718
754
765
777
781
783
784
819
854
859
867
874
907
1009
1035
1042
1060
1086
1114
1124
1128
1171
1258
1286
1327
1383
1416
1459
1467
1483
1502
1539
1581
1618
1694
1702
1743
1781
1784
1789
1799
1807
1809
1825
1842
1869
1913
1945
1953
2046
2066
2073
2089
2099
2136
2154
2182
2213
2233
2238
2294
2310
2352
2388
2389
2407
2417
2482
2488
2536
2541
2551
2566
2595
2604
2616
2623
2643
2652
2656
2667
2681
2690
2715
2747
2771
2786
2800
2820
2834
2843
2847
2853
2854
2875
2911
2913
2933
2934
3086
3090
3098
3114
3151
3153
3168
3187
3230
3253
3284
3300
3393
3398
3437
3474
3490
3537
3556
3561
3587
3623
3638
3651
3709
3750
3778
3787
3827
3847
3869
3880
3914
3919
3931
3995
4021
4022
4052
4059
4127
4147
4197
4209
4225
4241
4277
4289
4298
4324
4336
4347
4359
4379
4404
4422
4449
4460
4482
4530
4533
4606
4608
4723
4789
4819
4835
4846
4871
4890
4945
4961
5057
5146
5173
5188
5193
5233
5234
5235
5241
5260
5267

In [200]:
def error_matrix(total_predict,y_test):
    predict_size = len(total_predict)
    #오차행렬 초기화
    tp = 0
    fp = 0
    tn = 0
    fn = 0 
    Accuracy = np.nan
    Recall = np.nan
    Percision = np.nan
    Specificity = np.nan
    for i in range(predict_size):
        modelP = 0
        modelN = 0
        trueP = 0
        trueN = 0
        # 모델에 대한 P,N
        equl =np.array_equal(total_predict[i],np.array([1,0,0,0,0,0]))
        if(equl==True): # 에러가 아니면
            modelN = 1
        else:
            modelP = 1
        #실제값에 대한 P,N
        equl =np.array_equal(y_test[i],np.array([1,0,0,0,0,0]))
        if(equl == True):
            trueN = 1
        else:
            trueP = 1
        #오차 행렬 업데이트
        if(modelP==1 and trueP == 1):
            tp += 1
        elif(modelP == 1 and trueN == 1):
            fp += 1
        elif(modelN == 1 and trueN == 1):
            tn += 1
        elif(modelN == 1 and trueP == 1):
            fn += 1
    if(tp+tn+fp+fn != 0):
        Accuracy = float(tp+tn)/float(tp+tn+fp+fn) # 전체 예측 중 맞게 예측
    if(tp+fn != 0):
        Recall = float(tp)/float(tp+fn) # 실제 오류 중 오류라고 예측한 것
    if(tp+fp != 0):
        Percision = float(tp)/float(tp+fp) # 오류라고 예측한 것 중 실제 오류
    if(fp+tn != 0):
        Specificity = float(tn)/float(fp+tn) # 오류가 아니라고 예측 한 것 중 정말 오류가 아닌 것
    
    return Accuracy,Recall,Percision,Specificity

In [201]:
error_matrix(total_predict,y_test)

(0.9927103615055497,
 0.9796219349543449,
 0.9826337522672601,
 0.9958486957216965)